## 연습문제 OrderEvent

(1) Solidity로 구현

블록체인에서 주문을 하고, 그 내역을 로컬 파일에 작성해 보자.

블록체인으로 주문을 전송하면, 이벤트가 발생하고 주문내역을 로컬파일에 쓰게 된다.

* ```EventTest.sol```에서는 이벤트가 발생하면서 그 로그를 파일에 작성한다 (```OrderEvent.txt```)

* ```OrderEvent.sol```을 수정하여:

	* ```order(상품항목, 주문 개수, 배송지)``` 함수 작성

	* ```getBalance()``` 함수 작성. 해당 프로그램 생성한 ```owner```만 읽을 수 있게 ```modfier isOnwer``` 작성.


(2) ```ganache```에서 배포하고, 다음과 같이 API를 사용해서 출력하게 프로그래밍하세요.

아래 로그가 발생하면 이벤트를 출력하고,

또한 파일에 쓰고, 주피터 노트북에서 그 파일을 출력하세요.

(```!cat src/OrderEvent.txt```, 윈도우에서는 ```cat``` 대신 ```type```)

```python
전송자주소, 1111, 3, 20 2-gil Hongji-dong Jongro-gu Seoul   
전송자주소, 1111, 5, 20 2-gil Hongji-dong Jongro-gu Seoul    
전송자주소, 1111, 20, 20 2-gil Hongji-dong Jongro-gu Seoul

```

(3) ganache 대신 geth로 위 (2)를 해보세요. (geth는 geth@8445 또는 geth@8446에서 자바스크립트로 한다는 의미)

복수 거래가 발생하므로, 한 건씩 발생하도록 'OrderEventUse.js' 파일을 나누어 코딩해도 된다.

### (1) Solidity로 구현

### 1. 개발

In [1]:
%%writefile ../src/OrderEvent.sol
pragma solidity >=0.5.0;

contract OrderEventTest {
    address owner;
    uint uintPrice = 10;
    event OrderLog(address _from, bytes2 _itemId, uint _value, string _addr);
    constructor() public {
        owner = msg.sender;
    }

    function order(bytes2 _itemId, uint quantity, string memory addr) public payable {
        uint256 orderAmount = quantity * uintPrice;
        require(msg.value == orderAmount);
        emit OrderLog(msg.sender, _itemId, msg.value, addr);
    }
    function getBalance() public view isOwner returns(uint){
        return address(this).balance;
    }
    modifier isOwner {
        require(msg.sender == owner);
        _;
    }
}

Writing ../src/OrderEvent.sol


### 2. 컴파일

In [3]:
!solc --abi --bin --gas ../src/OrderEvent.sol


======= ../src/OrderEvent.sol:OrderEventTest =======
Gas estimation:
construction:
   41074 + 159400 = 200474
external:
   getBalance():	1063
   order(bytes2,uint256,string):	infinite
Binary:
6080604052600a60015534801561001557600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555061031d806100656000396000f3fe6080604052600436106100295760003560e01c806312065fe01461002e5780637a59a92914610059575b600080fd5b34801561003a57600080fd5b50610043610149565b6040518082815260200191505060405180910390f35b6101476004803603606081101561006f57600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100c157600080fd5b8201836020820111156100d357600080fd5b803590602001918460018302840111640100000000831117156100f557600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f8201169

### (2) ganache

### 배포

In [4]:
%%writefile ../src/OrderEventDeploy.js
var Web3 = require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"_from","type":"address"},{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"},{"indexed":false,"internalType":"string","name":"_addr","type":"string"}],"name":"OrderLog","type":"event"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"internalType":"uint256","name":"quantity","type":"uint256"},{"internalType":"string","name":"addr","type":"string"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"}];
var _bin = "0x"+"6080604052600a60015534801561001557600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555061031d806100656000396000f3fe6080604052600436106100295760003560e01c806312065fe01461002e5780637a59a92914610059575b600080fd5b34801561003a57600080fd5b50610043610149565b6040518082815260200191505060405180910390f35b6101476004803603606081101561006f57600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100c157600080fd5b8201836020820111156100d357600080fd5b803590602001918460018302840111640100000000831117156100f557600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f8201169050808301925050505050505091929192905050506101aa565b005b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16146101a457600080fd5b47905090565b6000600154830290508034146101bf57600080fd5b7f0cee52e22e6b387c4183ee502522fa33f030e118cc6065c7a842e8b7c00e4a6a33853485604051808573ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001847dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff19167dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916815260200183815260200180602001828103825283818151815260200191508051906020019080838360005b838110156102a4578082015181840152602081019050610289565b50505050905090810190601f1680156102d15780820380516001836020036101000a031916815260200191505b509550505050505060405180910390a15050505056fea2646970667358221220d98714de9118985195ae2b75c2fe6c7fb35287bf0062c195223adaea3b59e15964736f6c63430006040033";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from "+accounts[0]);
    var deployed = await new web3.eth.Contract(_abi)
        .deploy({data:_bin})
        .send({from:accounts[0], gas:1000000}, function(err, transactionHash) {
            if (!err)
                console.log("hash: "+transactionHash);
        });
    console.log("The contract deployed to: "+deployed.options.address)
}
deploy()

Writing ../src/OrderEventDeploy.js


In [22]:
!node ../src/OrderEventDeploy.js

Deploying the contract from 0x5743d8a1fa0395AC566abaFab53D73aD8bCFe1de
hash: 0xcfed415bba173fd5baec97ae478001a9634be4d56b67d190744fcfce9e350ee1
The contract deployed to: 0xe4dCf8989653b98715aF2534D3633204f341cd1e


### 사용

In [28]:
%%writefile ../src/OrderEventUse.js
var Web3 = require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
}
var fs = require('fs');

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"_from","type":"address"},{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"},{"indexed":false,"internalType":"string","name":"_addr","type":"string"}],"name":"OrderLog","type":"event"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"internalType":"uint256","name":"quantity","type":"uint256"},{"internalType":"string","name":"addr","type":"string"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"}];

async function doIt() {
    var _contract = new web3.eth.Contract(_abi, "0xe4dCf8989653b98715aF2534D3633204f341cd1e");
    
    var event = _contract.events.OrderLog({fromBlock: 39}, function (err, ret) {
        if (!err) {
            log = JSON.stringify(ret.returnValues);
            console.log("====================================================")
            console.log("Event Fired: "+log);
            newLog = JSON.stringify(ret.returnValues._from)+", "+JSON.stringify(ret.returnValues._itemId)
                        +", "+JSON.stringify(ret.returnValues._value)+", "+JSON.stringify(ret.returnValues._addr)+"\n"
            fs.appendFile("../src/OrderEvent.txt", newLog, 'utf-8', function (e) {
                if (!e) {
                    console.log(">> Writing to file");
                }
            });
        }
    });
    
    const accounts = await web3.eth.getAccounts();
    my = await _contract.methods.order("0x1111", 3, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from:accounts[0], gas:100000, value:30});
    console.log("\n---> function called "+JSON.stringify(my.events.OrderLog.returnValues));
    
    my = await _contract.methods.order("0x1111", 5, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from:accounts[0], gas:100000, value:50});
    console.log("\n---> function called "+JSON.stringify(my.events.OrderLog.returnValues));
    
    my = await _contract.methods.order("0x1111", 20, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from:accounts[0], gas:100000, value:200});
    console.log("\n---> function called "+JSON.stringify(my.events.OrderLog.returnValues));
    
    process.exit(1);
}
doIt()

Overwriting ../src/OrderEventUse.js


In [29]:
!node ../src/OrderEventUse.js

Event Fired: {"0":"0x5743d8a1fa0395AC566abaFab53D73aD8bCFe1de","1":"0x1111","2":"30","3":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x5743d8a1fa0395AC566abaFab53D73aD8bCFe1de","_itemId":"0x1111","_value":"30","_addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
>> Writing to file

---> function called {"0":"0x5743d8a1fa0395AC566abaFab53D73aD8bCFe1de","1":"0x1111","2":"30","3":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x5743d8a1fa0395AC566abaFab53D73aD8bCFe1de","_itemId":"0x1111","_value":"30","_addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
Event Fired: {"0":"0x5743d8a1fa0395AC566abaFab53D73aD8bCFe1de","1":"0x1111","2":"50","3":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x5743d8a1fa0395AC566abaFab53D73aD8bCFe1de","_itemId":"0x1111","_value":"50","_addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
>> Writing to file

---> function called {"0":"0x5743d8a1fa0395AC566abaFab53D73aD8bCFe1de","1":"0x1111","2":"50","3":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x5743d8a1fa0395AC566

### 작성된 log 출력

In [30]:
!type ..\src\OrderEvent.txt

"0x5743d8a1fa0395AC566abaFab53D73aD8bCFe1de", "0x1111", "30", "20 2-gil Hongji-dong Jongro-gu Seoul"
"0x5743d8a1fa0395AC566abaFab53D73aD8bCFe1de", "0x1111", "50", "20 2-gil Hongji-dong Jongro-gu Seoul"
"0x5743d8a1fa0395AC566abaFab53D73aD8bCFe1de", "0x1111", "200", "20 2-gil Hongji-dong Jongro-gu Seoul"


### (3) geth @8445

### 배포

In [31]:
%%writefile ../src/OrderEventDeploy-geth.js
var Web3 = require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
}

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"_from","type":"address"},{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"},{"indexed":false,"internalType":"string","name":"_addr","type":"string"}],"name":"OrderLog","type":"event"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"internalType":"uint256","name":"quantity","type":"uint256"},{"internalType":"string","name":"addr","type":"string"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"}];
var _bin = "0x"+"6080604052600a60015534801561001557600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555061031d806100656000396000f3fe6080604052600436106100295760003560e01c806312065fe01461002e5780637a59a92914610059575b600080fd5b34801561003a57600080fd5b50610043610149565b6040518082815260200191505060405180910390f35b6101476004803603606081101561006f57600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100c157600080fd5b8201836020820111156100d357600080fd5b803590602001918460018302840111640100000000831117156100f557600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f8201169050808301925050505050505091929192905050506101aa565b005b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16146101a457600080fd5b47905090565b6000600154830290508034146101bf57600080fd5b7f0cee52e22e6b387c4183ee502522fa33f030e118cc6065c7a842e8b7c00e4a6a33853485604051808573ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001847dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff19167dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916815260200183815260200180602001828103825283818151815260200191508051906020019080838360005b838110156102a4578082015181840152602081019050610289565b50505050905090810190601f1680156102d15780820380516001836020036101000a031916815260200191505b509550505050505060405180910390a15050505056fea2646970667358221220d98714de9118985195ae2b75c2fe6c7fb35287bf0062c195223adaea3b59e15964736f6c63430006040033";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from "+accounts[0]);
    var deployed = await new web3.eth.Contract(_abi)
        .deploy({data:_bin})
        .send({from:accounts[0], gas:1000000}, function(err, transactionHash) {
            if (!err)
                console.log("hash: "+transactionHash);
        });
    console.log("The contract deployed to: "+deployed.options.address)
}
deploy()

Writing ../src/OrderEventDeploy-geth.js


In [55]:
!node ../src/OrderEventDeploy-geth.js

Deploying the contract from 0x74cCD5B995B3480fE9063EE539EBF065385F61C8
hash: 0xdbfc330fe63a1b6fd26a1542217c5f48ba33d6d0dd9acccc5dc68e2e3b4fd8a1
The contract deployed to: 0x984810E666dC45f06E1452163fe935c2b76002d9


### 사용

In [74]:
%%writefile ../src/OrderEventUse-geth.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8446"));
var fs = require('fs');

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"_from","type":"address"},{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"},{"indexed":false,"internalType":"string","name":"_addr","type":"string"}],"name":"OrderLog","type":"event"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"internalType":"uint256","name":"quantity","type":"uint256"},{"internalType":"string","name":"addr","type":"string"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"}];

async function doIt() {
    
    var _contract = new web3.eth.Contract(_abi, "0x984810E666dC45f06E1452163fe935c2b76002d9");
    
    var event = _contract.events.OrderLog({fromBlock:0}, function (err, ret) {
        if (!err) {
            log = JSON.stringify(ret.returnValues);
            console.log("====================================================")
            console.log("Event Fired: "+log);
            newLog = JSON.stringify(ret.returnValues._from)+", "+JSON.stringify(ret.returnValues._itemId)
                        +", "+JSON.stringify(ret.returnValues._value)+", "+JSON.stringify(ret.returnValues._addr)+"\n"
            fs.appendFile("../src/OrderEvent-geth.txt", newLog, 'utf-8', function (e) {
                if (!e) {
                    console.log(">> Writing to file");
                }
            });
        }
    });
    
    const accounts = await web3.eth.getAccounts();
    my = await _contract.methods.order("0x1111", 3, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from:accounts[0], gas:100000, value:30});
    console.log("\n---> function called "+JSON.stringify(my.events.OrderLog.returnValues));
    
    my = await _contract.methods.order("0x1111", 5, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from:accounts[0], gas:100000, value:50});
    console.log("\n---> function called "+JSON.stringify(my.events.OrderLog.returnValues));
    
    my = await _contract.methods.order("0x1111", 20, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from:accounts[0], gas:100000, value:200});
    console.log("\n---> function called "+JSON.stringify(my.events.OrderLog.returnValues));
    
    process.exit(1);
}
doIt()

Overwriting ../src/OrderEventUse-geth.js


personal.unlockAccount(eth.accounts[0], "password", 0); 실행

In [59]:
!node ../src/OrderEventUse-geth.js

^C


jupyter notebook에서 실행되지 않아 단말에서 실행했지만, 해당 오류를 해결하지 못했습니다. 

단말에 하나씩 실행해 본 결과, `order 함수`는 실행이 잘 되지만 설정해놓은 event 호출 자체가 되지 않아 반환값인 `returnValues`에 값이 들어가지 못하는 것 같습니다. 저번 과제였던 `Multiply7`도 ganache에서는 실행이 잘 되지만 geth에서 실행 시 같은 오류가 발생합니다. 해당 오류는 더 찾아보도록 하겠습니다. 죄송합니다. 

(node:29364) UnhandledPromiseRejectionWarning: TypeError: Cannot read property 'returnValues' of undefined
    at doIt (C:\GitHubRepos\Code\201710928\src\3.js:22:77)
    at processTicksAndRejections (internal/process/task_queues.js:93:5)
(Use `node --trace-warnings ...` to show where the warning was created)
(node:29364) UnhandledPromiseRejectionWarning: Unhandled promise rejection. This error originated either by throwing inside of an async function without a catch block, or by rejecting a promise which was not handled with .catch(). To terminate the node process on unhandled promise rejection, use the CLI flag `--unhandled-rejections=strict` (see https://nodejs.org/api/cli.html#cli_unhandled_rejections_mode). (rejection id: 1)
(node:29364) [DEP0018] DeprecationWarning: Unhandled promise rejections are deprecated. In the future, promise rejections that are not handled will terminate the Node.js process with a non-zero exit code.